# 02: Model

In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers.legacy import Adam
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback

import tensorflow_privacy
# from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras
from tensorflow_privacy.privacy.optimizers import dp_optimizer
from tensorflow_privacy.privacy.optimizers import dp_optimizer_vectorized
from tensorflow_privacy.privacy.optimizers import dp_optimizer_keras_vectorized
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

#from tensorflow.keras.optimizers.legacy import Adam, AdamW

from synthesizers.cgan.model import ConditionalGAN, GANMonitor
from synthesizers.preprocessing.wesad import Subject, WESADDataset
from synthesizers.utils.training import data_split, buildDataCTST, generate_and_plot_data, synthetic_dataset, get_optimizer


In [5]:
DATA_PATH = 'data/wesad/wesad_preprocessed_1hz.csv'
SAMPLING_RATE = 1

In [6]:
df = pd.read_csv(DATA_PATH, index_col=0)
df_label = df['label']
df_stress = df[df['label']==1]
df_no_stress = df[df['label']==0]

In [7]:
trainX, label_trainX = WESADDataset.create_windows(df=df, fs=1)
mos, _ = WESADDataset.create_windows(df_stress, SAMPLING_RATE)
non_mos, _ = WESADDataset.create_windows(df_no_stress,SAMPLING_RATE)

trainX = np.delete(trainX, 6, axis=2)
mos = np.delete(mos, 6, axis=2)
non_mos = np.delete(non_mos, 6, axis=2)

In [8]:
num_split = 0.8
trainmos, testmos = data_split(mos, num_split)
trainnomos, testnomos = data_split(non_mos, num_split)

print(trainmos.shape)
print(testmos.shape)
print(trainnomos.shape)
print(testnomos.shape)

(132, 60, 6)
(34, 60, 6)
(308, 60, 6)
(78, 60, 6)


In [9]:
# Training Hyperparameters
DP_TRAINING = False
NUM_FEATURES = trainX.shape[2]
SEQ_LENGTH = 60
LATENT_DIM = SEQ_LENGTH
BATCH_SIZE = 32
HIDDEN_UNITS = 512
EPOCHS = 3000
ACTIVATION = "relu"
RANDOM_SEED = 1

# DP Training Hyperparameter
L2_NORM_CLIP = 1.5
NOISE_MULTIPLIER = 1.3
NUM_MICROBATCHES = 250
DP_LEARNING_RATE = 0.25

# Start a run, tracking hyperparameters
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="cgan_sweep",
#     # track hyperparameters and run metadata with wandb.config
#     # config={
#     #     "layer_1": 512,
#     #     "layer_2": 10,
#     #     "hidden_units": HIDDEN_UNITS,
#     #     "epoch": EPOCHS,
#     #     "batch_size": BATCH_SIZE
#     # }
# )

# Define run config
config = {
    "activation_function": ACTIVATION,
    "hidden_units": HIDDEN_UNITS,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "random_seed": RANDOM_SEED,
    "num_features": NUM_FEATURES,
    "seq_length": SEQ_LENGTH,
    "dp_training": DP_TRAINING
}



Sweep Config only for sweep case

In [10]:
# Define sweep config
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'c2st_score'},
    'parameters': 
    {
        #'batch_size': {'values': [32]},
        #'d_lr': {'max': 0.1, 'min': 0.0001},
        #'g_lr': {'max': 0.1, 'min': 0.0001},
        #'hidden_units': {'values': [16, 32, 64, 128, 256]},
        # 'filter1': {'values': [32, 64]},
        # 'filter2': {'values': [64, 128]},
        # 'filter3': {'values': [32, 64]},
        'activation_function': {'values': ['sigmoid', 'tanh', 'relu', 'linear']},
        #'kernel_size1': {'values': [3, 5, 7]},
        #'kernel_size2': {'values': [3, 5, 7]},
        #'kernel_size3': {'values': [3, 5, 7]},
        #'optimizer': {
        #    'values': ['adam',]#'adamw', 'sgd']
        #},
    }
}

# Initialize sweep by passing in config. 
# (Optional) Provide a name of the project.
sweep_id = wandb.sweep(
  sweep=sweep_configuration, 
  project='cgan_sweep'
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 42ra3vjd
Sweep URL: https://wandb.ai/nw20hewo/cgan_sweep/sweeps/42ra3vjd


In [11]:
# load dataset into tf dataset
dataset = tf.data.Dataset.from_tensor_slices((trainX, label_trainX))
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)
dataset

tf.random.set_seed(RANDOM_SEED)
randomTrainMos = tf.random.normal(shape=(trainmos.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
randomTrainNoMos = tf.random.normal(shape=(trainnomos.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
randomTestMos = tf.random.normal(shape=(testmos.shape[0], LATENT_DIM))

tf.random.set_seed(RANDOM_SEED)
randomTestNoMos = tf.random.normal(shape=(testnomos.shape[0], LATENT_DIM))

In [13]:
def main():

    run = wandb.init(
        project="cgan",
        config=config
    )

    cond_gan = ConditionalGAN(
        num_features=NUM_FEATURES,
        seq_length=SEQ_LENGTH,
        latent_dim=LATENT_DIM,
        discriminator=ConditionalGAN.conditional_discriminator(
            seq_length=SEQ_LENGTH, 
            num_features=NUM_FEATURES,
            filters=[32, 64, 32],
            #filters=[wandb.config.filter1, wandb.config.filter2, wandb.config.filter3],
            #kernel_sizes=[wandb.config.kernel_size1, wandb.config.kernel_size2, wandb.config.kernel_size3]
            ),
        generator=ConditionalGAN.conditional_generator(
            hidden_units=SEQ_LENGTH, 
            seq_length=SEQ_LENGTH, 
            latent_dim=LATENT_DIM,
            num_features=NUM_FEATURES,
            activation_function= ACTIVATION
        )
    )
    if DP_TRAINING:

        d_optimizer = dp_optimizer_vectorized.VectorizedDPAdamOptimizer( #vectorized adam am schnellsten
            l2_norm_clip=L2_NORM_CLIP,
            noise_multiplier=NOISE_MULTIPLIER,
            num_microbatches=NUM_MICROBATCHES,
            learning_rate=DP_LEARNING_RATE
        )
    else:
        d_optimizer = Adam(learning_rate=0.0002, beta_1=0.5) # get_optimizer(0.0002, wandb.config.optimizer)#

    g_optimizer = Adam(learning_rate=0.0002, beta_1=0.5) # get_optimizer(0.0002, wandb.config.optimizer)#

    cond_gan.compile(
        d_optimizer= d_optimizer, # Adam(learning_rate=0.0002, beta_1=0.5),
        g_optimizer= g_optimizer, # Adam(learning_rate=0.0002, beta_1=0.5), #optimizer
        loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    )

    print(f"{cond_gan.d_optimizer} is used")

    if DP_TRAINING:
        generator_save_path = f"models/dp/{EPOCHS}/cgan_generator/"
    else:
        generator_save_path = f"models/no_dp/{EPOCHS}/cgan_generator"

    logger_callback = WandbCallback()

    history = cond_gan.fit(
        dataset,
        epochs=EPOCHS,
        callbacks=[
            GANMonitor(
                trainmos,
                trainnomos,
                testmos,
                testnomos,
                randomTrainMos,
                randomTrainNoMos,
                randomTestMos,
                randomTestNoMos,
                num_seq=50,
                save_path=generator_save_path,
                batch_size=BATCH_SIZE,
                seq_length=SEQ_LENGTH,
                num_features=NUM_FEATURES,
            ),
            logger_callback
        ],
    )

    if DP_TRAINING:
        base_path = f"models/dp/{EPOCHS}/"
        cond_gan.generator.save(f"{base_path}cgan_generator")
        cond_gan.discriminator.save(f"{base_path}cgan_discriminator")
    else:
        base_path = f"models/no_dp/{EPOCHS}/"
        cond_gan.generator.save(f"{base_path}cgan_generator")
        cond_gan.discriminator.save(f"{base_path}cgan_discriminator")

wandb.login()
main()
# wandb.agent(sweep_id, function=main)

wandb: Currently logged in as: nw20hewo. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


<keras.optimizers.legacy.adam.Adam object at 0x2988ef340> is used
Epoch 1/3000


2023-06-08 21:56:20.435951: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17/18 [===========================>..] - ETA: 0s - g_loss: 0.6066 - d_loss: 0.7171 - div_term: 0.1092[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 264 samples in 0.000s...
[t-SNE] Computed neighbors for 264 samples in 0.018s...
[t-SNE] Computed conditional probabilities for sample 264 / 264
[t-SNE] Mean sigma: 0.105204
[t-SNE] KL divergence after 250 iterations with early exaggeration: 42.408073
[t-SNE] KL divergence after 300 iterations: 0.345229
18/18 [==============================] - 6s 171ms/step - g_loss: 0.6038 - d_loss: 0.7135 - div_term: 0.1151
Epoch 2/3000
18/18 [==============================] - 1s 50ms/step - g_loss: 0.3316 - d_loss: 0.6239 - div_term: 0.4765
Epoch 3/3000
18/18 [==============================] - 1s 52ms/step - g_loss: -0.0798 - d_loss: 0.7002 - div_term: 0.7635
Epoch 4/3000
18/18 [==============================] - 1s 53ms/step - g_loss: 0.0781 - d_loss: 0.6480 - div_term: 0.6574
Epoch 5/3000
18/18 [==============================] - 1s 51ms/step

/Users/nils/thesis/Data_Generation/metric/visualization.py:75: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.



[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 264 samples in 0.000s...
[t-SNE] Computed neighbors for 264 samples in 0.028s...
[t-SNE] Computed conditional probabilities for sample 264 / 264
[t-SNE] Mean sigma: 0.274032
[t-SNE] KL divergence after 250 iterations with early exaggeration: 46.488274
[t-SNE] KL divergence after 300 iterations: 0.177973
18/18 [==============================] - 2s 130ms/step - g_loss: -0.9066 - d_loss: 0.6394 - div_term: 1.7607
Epoch 1002/3000
18/18 [==============================] - 1s 58ms/step - g_loss: -0.8137 - d_loss: 0.6234 - div_term: 1.6782
Epoch 1003/3000
18/18 [==============================] - 1s 65ms/step - g_loss: -0.9697 - d_loss: 0.6287 - div_term: 1.7269
Epoch 1004/3000
18/18 [==============================] - 1s 59ms/step - g_loss: -0.8858 - d_loss: 0.6274 - div_term: 1.7532
Epoch 1005/3000
18/18 [==============================] - 1s 58ms/step - g_loss: -0.9741 - d_loss: 0.6254 - div_term: 1.7659
Epoch 1006/3000
18/18 [========

In [ ]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=trainX.shape[0],
                                              batch_size=BATCH_SIZE,
                                              noise_multiplier=NOISE_MULTIPLIER,
                                              epochs=EPOCHS,
                                              delta=1e-5)


DP-SGD with sampling rate = 5.8% and noise_multiplier = 1.3 iterated over 173 steps satisfies differential privacy with eps = 3.61 and delta = 1e-05.
The optimal RDP order is 6.0.


(3.6056087409381825, 6.0)